# Introduction to Memory Profiling

> Objectives:
> * Be introduced to memory profiling using different tools
> * Some small introduction to time profiling in IPython too


## ipython_memwatcher

Our recommended way to profile memory consumption for this tutorial will be [ipython_memwatcher](https://pypi.python.org/pypi/ipython_memwatcher):


In [1]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [1] used 0.027 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 34.961 MiB


In [2]:
# Let's create a big object
a = [i for i in range(1000*1000)]

In [2] used 31.117 MiB RAM in 0.080s, peaked 0.000 MiB above current, total RAM usage 66.078 MiB


In [3]:
# Get some measurements from the last executed cell:
meas = mw.measurements
meas

Measurements(memory_delta=31.1171875, time_delta=0.08034610748291016, memory_peak=0, memory_usage=66.078125)

In [3] used 0.031 MiB RAM in 0.009s, peaked 0.000 MiB above current, total RAM usage 66.109 MiB


In [4]:
# MemWatcher.measurements is a named tuple.  We can easily get info out of it:
meas.memory_delta

31.1171875

In [4] used 0.004 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 66.113 MiB


In [5]:
# This takes between 32 ~ 35 bytes per element:
meas.memory_delta * (2**20) / 1e6

32.628736

In [5] used 0.004 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 66.117 MiB


In [6]:
# What are these elements made from?
type(a[0])

int

In [6] used 0.000 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 66.117 MiB


In [7]:
# How much memory take an int?
# Beware: the size below will depend on whether you are using a 32-bit or 64-bit Python
import sys
sys.getsizeof(a[0])

24

In [7] used 0.012 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 66.129 MiB


Ok.  On 64-bits, that means that the int object allocates 8 bytes for the integer value, and 16 bytes for other metadata (Python object).  But 24 is quite less than 32~35.  Where this overhead comes from?

Well, it turns out that the list structure needs additional pointers to reference the different components.

![List diagram](http://www.brpreiss.com/books/opus7/html/img579.gif)

## memory_profiler

[memory_profiler](https://pypi.python.org/pypi/memory_profiler) is a basic module for memory profiling that many others use (like the `ipython_memwatcher` above) and it interacts well with ipython, so it is worth to see how it works:

In [8]:
%load_ext memory_profiler

In [8] used 0.016 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 66.145 MiB


In [9]:
# Use %memit magic command exposed by memory_profiler
%memit b = [i for i in range(1000*1000)]

peak memory: 137.19 MiB, increment: 71.03 MiB
In [9] used 71.180 MiB RAM in 0.373s, peaked 0.000 MiB above current, total RAM usage 137.324 MiB


Please note that the `peak_memory` in this case is different than the `peaked_memory` reported by ipython_memwatcher package.

## Guppy

Guppy is nice for having an overview of how different structures are using our memory:

In [10]:
from guppy import hpy; hp=hpy()
hp.heap()

Partition of a set of 2146590 objects. Total size = 84885088 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0 2003288  93 48078912  57  48078912  57 int
     1   1815   0 16501560  19  64580472  76 list
     2  69581   3  6530288   8  71110760  84 str
     3  33717   2  2888792   3  73999552  87 tuple
     4   1815   0  1798440   2  75797992  89 dict (no owner)
     5    480   0  1405440   2  77203432  91 dict of module
     6   8649   0  1107072   1  78310504  92 types.CodeType
     7   8424   0  1010880   1  79321384  93 function
     8   1021   0   920176   1  80241560  95 type
     9   1021   0   835000   1  81076560  96 dict of type
<657 more rows. Type e.g. '_.more' to view.>

In [10] used 54.395 MiB RAM in 3.354s, peaked 0.000 MiB above current, total RAM usage 191.719 MiB


In [11]:
del b
hp.heap()

Partition of a set of 1146865 objects. Total size = 52770672 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0 1003546  88 24085104  46  24085104  46 int
     1   1812   0  8374880  16  32459984  62 list
     2  69586   6  6530608  12  38990592  74 str
     3  33718   3  2888896   5  41879488  79 tuple
     4   1821   0  1801656   3  43681144  83 dict (no owner)
     5    480   0  1405440   3  45086584  85 dict of module
     6   8649   1  1107072   2  46193656  88 types.CodeType
     7   8423   1  1010760   2  47204416  89 function
     8   1021   0   920176   2  48124592  91 type
     9   1021   0   835000   2  48959592  93 dict of type
<653 more rows. Type e.g. '_.more' to view.>

In [11] used 0.867 MiB RAM in 0.723s, peaked 0.000 MiB above current, total RAM usage 192.586 MiB


In [12]:
# Size of the list (beware, this does not include the contents!)
hp.iso(a)

Partition of a set of 1 object. Total size = 8126536 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      1 100  8126536 100   8126536 100 list

In [12] used 0.000 MiB RAM in 0.003s, peaked 0.000 MiB above current, total RAM usage 192.586 MiB


## %time and %timeit

In [13]:
# IPython provides a magic command to see how much time a command takes to run
%time asum = sum(a)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 7.29 ms
In [13] used 0.000 MiB RAM in 0.009s, peaked 0.000 MiB above current, total RAM usage 192.586 MiB


Note that `%time` offers quite detailed statistics on the time spent.

Also, the time reported by MemoryWatcher has a typical overhead of 1~5 ms over the time reported by %time, so when the times to measure are about this order then it is better to rely on the %time (or %timeit below) values.  

In [14]:
# We have another way to measure timings doing several loops and getting the mean
%timeit bsum = sum(a)
# However, one must notice that %timeit does not return the result of expressions

100 loops, best of 3: 7.53 ms per loop
In [14] used 0.000 MiB RAM in 3.215s, peaked 0.000 MiB above current, total RAM usage 192.586 MiB


Interestingly, %timeit allows to retrieve the measured times in loops with the -o flag:

In [15]:
t = %timeit -o sum(a)
print(t.all_runs)
print(t.best)

100 loops, best of 3: 6.74 ms per loop
[0.6753439903259277, 0.6742100715637207, 0.6783020496368408]
0.00674210071564
In [15] used 0.000 MiB RAM in 2.789s, peaked 0.000 MiB above current, total RAM usage 192.586 MiB


And one can specify the number of loops (-n) and the number of repetitions (-r):

In [16]:
t = %timeit -r1 -n1 -o sum(a)
print(t.all_runs)
print(t.best)

1 loop, best of 1: 10.2 ms per loop
[0.010194778442382812]
0.0101947784424
In [16] used 0.000 MiB RAM in 0.014s, peaked 0.000 MiB above current, total RAM usage 192.586 MiB


### Exercise 1

Provided a dictionary like:

```
d = dict(("key: %i"%i, i*2) for i in a)
```

Try to guess how much RAM it uses using the techniques introduced above.

Why do you think it takes more space than a list?

*Hint*: Every entry in a dictionary has pointers to two objects: key and value. 

## Solution

In [17]:
d = dict(("key: %i"%i, i*2) for i in a)

In [17] used 101.176 MiB RAM in 0.849s, peaked 0.000 MiB above current, total RAM usage 293.762 MiB


In [18]:
# Compute the size of key + value
sys.getsizeof("key: 100000") + sys.getsizeof(1)

72

In [18] used 0.008 MiB RAM in 0.002s, peaked 0.000 MiB above current, total RAM usage 293.770 MiB


In [19]:
# Compute the size of the dict struct (using guppy here)
hp.iso(d)

Partition of a set of 1 object. Total size = 50331928 bytes.
 Index  Count   %     Size   % Cumulative  % Kind (class / dict of class)
     0      1 100 50331928 100  50331928 100 dict (no owner)

In [19] used 0.062 MiB RAM in 0.587s, peaked 0.000 MiB above current, total RAM usage 293.832 MiB


So, guppy is telling us that just the dictionary structure is taking ~50 MB, whereas the contents alone are taking ~70MB, so we should have expected the dictionary to consume ~120 MB.  However, our `MemWatcher` instance is reporting just ~100 MB.  Take away lesson: measuring memory consumption in Python is tricky but using proper tools we can still get valuable hints!